In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt

torch.cuda.is_available()

False

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# data to test code
# !tar xvzf /content/drive/MyDrive/GP_project/part3.tar.gz

In [4]:
##export pytorch model to onnx 
# ! pip install onnxruntime
# !pip --quiet install onnx onnxruntime onnxsim
# !pip install onnx-tf
# !git clone https://github.com/deepcam-cn/yolov5-face.git
# %cd /content/yolov5-face
# !git clone https://github.com/hpc203/yolov5-face-landmarks-opencv-v2.git
# !python /content/yolov5-face/main_export_onnx.py --cfg /content/yolov5-face/models/yolov5s.yaml --weights /content/drive/MyDrive/GP_project/Weights/yolov5s-face.pt --image /content/yolov5-face-landmarks-opencv-v2/selfie.jpg  


In [5]:
# !pip3 install --upgrade opencv-python
# print(cv2. __version__ )

In [6]:
## to copy model file from drive to notebook
# %cp /content/drive/MyDrive/GP_project/Weights/yolov5s-face.onnx /content/yolov5s-face.onnx

**Run onnx model using opencv2**




In [7]:
def _make_grid(nx=20, ny=20):
        xv, yv = np.meshgrid(np.arange(ny), np.arange(nx))
        return np.stack((xv, yv), 2).reshape((-1, 2)).astype(np.float32)
        


def drawPred(frame, conf, left, top, right, bottom, landmark):
      """plot predicted bounding box and facial landmarks on the image"""
      # Draw a bounding box.
      cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), thickness=2)
      label = '%.2f' % conf
      #Display the label at the top of the bounding box
      cv2.putText(frame,label,(left, top - 2),cv2.FONT_HERSHEY_SIMPLEX,0.75,[225, 255, 255],thickness=2) 
      for i in range(5):
          cv2.circle(frame, (landmark[i*2], landmark[i*2+1]), 1, (0,255,0), thickness=-1)
      return frame 


def align_faces(im,bbox, landmark=None):
    """align predicted face using 5 facial landmarks 
       and crop it with shape (112,112).
       if landmarks don't exis, we crop the image using bounding box """
    M = None
    warped=None
    x1 = int(bbox[0])  # rect.left()
    y1 = int(bbox[1])  # rect.top()
    x2 = int(bbox[2])  # rect.right()
    y2 = int(bbox[3])   # rect.bottom()
    
    if landmark is not None:
        src = np.array([
          [30.2946, 51.6963],
          [65.5318, 51.5014],
          [48.0252, 71.7366],
          [33.5493, 92.3655],
          [62.7299, 92.2041] ], dtype=np.float32 )
        src[:,0] += 8.0
        dst = landmark.astype(np.float32)
        M = cv2.estimateAffine2D(dst,src)[0]
        warped = cv2.warpAffine(im,M,(112,112), borderValue = 0.0)
    else:
        warped = im[y1:y1+y2, x1:x1+x2]
    return warped


In [8]:
def non_max_supression(frame, outs ,confThreshold=0.3,nmsThreshold=0.45):
        frameHeight = frame.shape[0]
        frameWidth = frame.shape[1]
        
        ratioh, ratiow = frameHeight / 640, frameWidth / 640
        # Scan through all the bounding boxes output from the network and keep only the
        # ones with high confidence scores. Assign the box's class label as the class with the highest score.

        confidences = []
        boxes = []
        landmarks = []
        #print(outs.shape)
        for detection in outs:
      
            #confidence = detection[15]
            conf=detection[4]*detection[15]
            #print(conf)
            
            if detection[4] > confThreshold:
                center_x = int(detection[0] * ratiow)
                center_y = int(detection[1] * ratioh)
                width = int(detection[2] * ratiow)
                height = int(detection[3] * ratioh)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)

                confidences.append(float(conf))
                boxes.append([left, top, width, height])
                landmark = detection[5:15] * np.tile(np.float32([ratiow,ratioh]), 5)
                landmarks.append(landmark.astype(np.int32))
        # Perform non maximum suppression to eliminate redundant overlapping boxes with
        # lower confidences.
        indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
        
        return np.array(boxes)[indices],np.array(landmarks)[indices],np.array(confidences)[indices]

In [9]:
def postprocess(frame,boxes,landmarks,confidences):
        """filter predicted bboxes, 
             if exist more than face in the image 
            ,take the closeset face to the camera using IOD metric
            and return image cropped and aligned.
            else if there is no faces predicted return None """
        im=frame.copy()
        #im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        max_iod=-1
        box_=None
        land_mark=None
        for box, landmark, confidence in zip(boxes, landmarks, confidences):
            lm = [tuple(landmark[i:i + 2]) for i in range(0, len(landmark), 2)]
            IOD = abs((lm[0][0] + lm[0][1] / 2.0) - (lm[1][0] + lm[1][1] / 2.0)) #This metric could be used to infer distance from the camera.
            if IOD > max_iod:  # filter bbox based on IOD(inter-ocular distance)
                  max_iod=IOD
                  box_=box
                  land_mark=lm 
            ## draw bounding box and landmarks       
            #frame = drawPred(frame, confidence, box[0], box[1], box[0] +  box[2], box[1] + box[3], landmark)
        if max_iod != -1:
            cropped=align_faces(im.copy(),box_, np.array(land_mark))
            # plt.imshow(cropped)
            # plt.show()
            # path='/content/cropped.jpg'
            # cv2.imwrite(path,cropped)    
            return cropped
        else:
           return None   

In [10]:
def process_anchors(prediction,inpWidth,inpHeight):
        anchors = [[4,5,  8,10,  13,16], [23,29,  43,55,  73,105], [146,217,  231,300,  335,433]]
        num_classes = 1
        nl = len(anchors)
        na = len(anchors[0]) // 2
        no = num_classes + 5 + 10
        grid = [np.zeros(1)] * nl
        stride = np.array([8., 16., 32.])
        anchor_grid = np.asarray(anchors, dtype=np.float32).reshape(nl, -1, 2)
        
        prediction[..., [0,1,2,3,4,15]] = 1 / (1 + np.exp(-prediction[..., [0,1,2,3,4,15]]))   ###sigmoid
        row_ind = 0
        for i in range(nl):
            h, w = int(inpHeight/stride[i]), int(inpWidth/stride[i])
            length = int(na * h * w)
            if grid[i].shape[2:4] != (h,w):
                grid[i] = _make_grid(w, h)
            
            g_i = np.tile(grid[i], (na, 1))
            a_g_i = np.repeat(anchor_grid[i], h * w, axis=0)
            prediction[row_ind:row_ind + length, 0:2] = (prediction[row_ind:row_ind + length, 0:2] * 2. - 0.5 + g_i) * int(stride[i])
            prediction[row_ind:row_ind + length, 2:4] = (prediction[row_ind:row_ind + length, 2:4] * 2) ** 2 * a_g_i

            prediction[row_ind:row_ind + length, 5:7] = prediction[row_ind:row_ind + length, 5:7] * a_g_i + g_i * int(stride[i])   # landmark x1 y1
            prediction[row_ind:row_ind + length, 7:9] = prediction[row_ind:row_ind + length, 7:9] * a_g_i + g_i * int(stride[i])  # landmark x2 y2
            prediction[row_ind:row_ind + length, 9:11] = prediction[row_ind:row_ind + length, 9:11] * a_g_i + g_i * int(stride[i])  # landmark x3 y3
            prediction[row_ind:row_ind + length, 11:13] = prediction[row_ind:row_ind + length, 11:13] * a_g_i + g_i * int(stride[i])  # landmark x4 y4
            prediction[row_ind:row_ind + length, 13:15] = prediction[row_ind:row_ind + length, 13:15] * a_g_i + g_i * int(stride[i])  # landmark x5 y5
            row_ind += length
        return prediction   


In [11]:
def detect(srcimg,model_path,inpWidth,inpHeight):
        
        #prepare input image
        blob = cv2.dnn.blobFromImage(srcimg, 1 / 255.0, (inpWidth, inpHeight), [0, 0, 0], swapRB=True, crop=False)
        # read model
        net = cv2.dnn.readNet(model_path)
        # Sets the input to the network
        net.setInput(blob)

        # Runs the forward pass to get output of the output layers
        outs = net.forward(net.getUnconnectedOutLayersNames())[0]
        #print(outs[0, [0,1,2,3,4,15]])
        # inference output
        outs=process_anchors(outs,inpWidth,inpHeight)
        
        return outs

In [12]:
def yolov5(srcimg,model_path='/kaggle/input/yolov5/yolov5s-face.onnx',confThreshold=0.3,nmsThreshold=0.45):
  #srcimg = cv2.imread(imgpath)
  # imname=imgpath.split('/')[-1]
  # save_path+='/'+imname
  dets = detect(srcimg,model_path,640,640)
  boxes,landmarks,confidences=non_max_supression(srcimg, dets)
  img = postprocess(srcimg,boxes,landmarks,confidences)
  return img
  # if img is not None:
  #   cv2.imwrite(save_path, img)
  # else:
  #   print("there is no faces in the image")    

In [13]:
# #test onnx model
# from tqdm import tqdm
# import os
# for img in tqdm(os.listdir('/content/part3')):
#     path = os.path.join('/content/part3', img)
#     img_data = cv2.imread(path) # BGR IMAGE
#     img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)
#     img=yolov5(img_data)

**convert to tflite model**

In [14]:
#export onnx model to tflite 
# !onnx-tf convert -i /content/yolov5s-face.onnx -o /content/
# converter = tf.lite.TFLiteConverter.from_saved_model('/content/')
# tflite_model = converter.convert()

# with open('/content/yolov5s_model.tflite', 'wb') as f:
#   f.write(tflite_model)

In [15]:
def detect_tf_model(srcimg,model_path,inpWidth,inpHeight):
   
   #prepare input     
   blob = cv2.dnn.blobFromImage(srcimg, 1 / 255.0, (inpWidth, inpHeight), [0, 0, 0], swapRB=True, crop=False)
   

   interpreter = tf.lite.Interpreter(model_path=model_path)
   #Allocate tensors.
   interpreter.allocate_tensors()
   # Get input and output tensors.
   input_details = interpreter.get_input_details()
   output_details = interpreter.get_output_details()

   # Test the model on random input data.
   input_shape = input_details[0]['shape']
   interpreter.set_tensor(input_details[0]['index'], blob)

   interpreter.invoke()

   # The function `get_tensor()` returns a copy of the tensor data.
   # Use `tensor()` in order to get a pointer to the tensor.
   output_data = interpreter.get_tensor(output_details[0]['index'])
   output_data=process_anchors(output_data,inpWidth,inpHeight)
  

   return output_data        


In [16]:
##test tflite model
# model_path='/content/yolov5s_model.tflite'
# imgpath='/content/3faces.PNG'
# srcimg = cv2.imread(imgpath)
# dets = detect_tf_model(srcimg,model_path,640,640)
# boxes,landmarks,confidences=non_max_supression(srcimg, dets)
# img = postprocess(srcimg,boxes,landmarks,confidences)
# if img is not None:
#     cv2.imwrite('/content/img.jpg', img)
# else:
#   print("there is no faces in the image") 